In [1]:
# If needed, uncomment to install deps
# !pip install -q transformers timm tifffile

import os
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import tifffile as tiff
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import SegformerForSemanticSegmentation


2025-09-04 07:06:07.776318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756969567.972814      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756969568.031640      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Dataset name: Sorted_3years_LULC

In [2]:
import os

print("Datasets under /kaggle/input/:")
print(os.listdir("/kaggle/input/"))

print("\nContents of your dataset folder:")
print(os.listdir("/kaggle/input/lulc-dataset"))


Datasets under /kaggle/input/:
['lulc-dataset']

Contents of your dataset folder:
['3yeardata']


In [7]:
import os

root_dir = "/kaggle/input/lulc-dataset/3yeardata"
print("Root contents:", os.listdir(root_dir))
# Check for subfolders
if "Images" in os.listdir(root_dir):
    print("Images folder contents:", os.listdir(os.path.join(root_dir, "Images"))[:10])
if "Mask" in os.listdir(root_dir):
    print("Mask folder contents:", os.listdir(os.path.join(root_dir,"Mask"))[:10])

Root contents: ['Images', 'Mask']
Images folder contents: ['Image_191.tif', 'Image_178.tif', 'Image_014.tif', 'Image_103.tif', 'Image_140.tif', 'Image_091.tif', 'Image_157.tif', 'Image_108.tif', 'Image_022.tif', 'Image_077.tif']
Mask folder contents: ['Mask_231.tif', 'Mask_073.tif', 'Mask_064.tif', 'Mask_036.tif', 'Mask_103.tif', 'Mask_065.tif', 'Mask_049.tif', 'Mask_143.tif', 'Mask_148.tif', 'Mask_141.tif']


In [20]:
!pip install -q transformers datasets huggingface_hub --no-deps


In [22]:
import os
from torch.utils.data import Dataset
from PIL import Image

class LULCDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform

        # Match files by sorting
        self.image_files = sorted(os.listdir(images_dir))
        self.mask_files  = sorted(os.listdir(masks_dir))

        assert len(self.image_files) == len(self.mask_files), \
            "❌ Number of images and masks do not match!"

        self.pairs = list(zip(self.image_files, self.mask_files))

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.pairs[idx][0])
        mask_path = os.path.join(self.masks_dir, self.pairs[idx][1])

        image = Image.open(img_path).convert("RGB")
        mask  = Image.open(mask_path).convert("L")  # grayscale

        if self.transform:
            image = self.transform(image)

        return image, mask


In [24]:
from transformers import SegformerImageProcessor

processor = SegformerImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")

dataset = LULCDataset(IMAGES_DIR, MASKS_DIR, transform=processor)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/image_processing_base.py:410: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type', 'reduce_labels'
  image_processor = cls(**image_processor_dict)


In [25]:
IMAGES_DIR = "/kaggle/input/lulc-dataset/3yeardata/Images"
MASKS_DIR  = "/kaggle/input/lulc-dataset/3yeardata/Mask"

dataset = LULCDataset(IMAGES_DIR, MASKS_DIR, processor)

print(f"✅ Dataset loaded with {len(dataset)} pairs")

# Show first 5 pairs to confirm correct matching
for i in range(5):
    img_file, mask_file = dataset.pairs[i]
    print(f"Pair {i}: {img_file}  <->  {mask_file}")


✅ Dataset loaded with 232 pairs
Pair 0: Image_001.tif  <->  Mask_001.tif
Pair 1: Image_002.tif  <->  Mask_002.tif
Pair 2: Image_003.tif  <->  Mask_003.tif
Pair 3: Image_004.tif  <->  Mask_004.tif
Pair 4: Image_005.tif  <->  Mask_005.tif


In [26]:
class SegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, image_size=512, num_classes=4, normalize=True):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.image_files = sorted([f for f in os.listdir(images_dir) if f.lower().endswith(('.tif','.tiff','.png','.jpg','.jpeg'))])
        self.mask_files  = sorted([f for f in os.listdir(masks_dir)  if f.lower().endswith(('.tif','.tiff','.png','.jpg','.jpeg'))])

        assert len(self.image_files) == len(self.mask_files), f"Images({len(self.image_files)}) != Masks({len(self.mask_files)})"
        self.image_size = image_size
        self.num_classes = num_classes
        self.normalize = normalize
        # ImageNet mean/std (SegFormer preproc defaults)
        self.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape(3,1,1)
        self.std  = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape(3,1,1)

    def __len__(self):
        return len(self.image_files)

    def _read_image(self, path):
        if path.lower().endswith(('.tif','.tiff')):
            arr = tiff.imread(path)          # may be float, extra dims
        else:
            arr = np.array(Image.open(path)) # HWC

        arr = np.squeeze(arr)
        # Scale non-uint8 to 0..255
        if arr.dtype != np.uint8:
            arr = (255 * (arr - arr.min()) / (arr.max() - arr.min() + 1e-8)).astype(np.uint8)
        # Grayscale → RGB
        if arr.ndim == 2:
            arr = np.stack([arr]*3, axis=-1)
        return arr  # HWC uint8

    def _read_mask(self, path):
        if path.lower().endswith(('.tif','.tiff')):
            arr = tiff.imread(path)
        else:
            arr = np.array(Image.open(path))
        arr = np.squeeze(arr)
        if arr.ndim == 3:
            arr = arr[:, :, 0]
        return arr  # HW (integers)

    def __getitem__(self, idx):
        img_path  = os.path.join(self.images_dir, self.image_files[idx])
        mask_path = os.path.join(self.masks_dir,  self.mask_files[idx])

        # ---- Image ----
        img = self._read_image(img_path)
        img = Image.fromarray(img).resize((self.image_size, self.image_size), resample=Image.BILINEAR)
        img = np.array(img, dtype=np.float32) / 255.0        # [0,1]
        img = np.transpose(img, (2, 0, 1))                   # C,H,W
        if self.normalize:
            img = (img - self.mean) / self.std

        # ---- Mask ----
        mask = self._read_mask(mask_path)
        mask = Image.fromarray(mask).resize((self.image_size, self.image_size), resample=Image.NEAREST)
        mask = np.array(mask, dtype=np.int64)
        # Remap anything >=4 to class 3 ("Others")
        mask[mask >= self.num_classes] = self.num_classes - 1

        return torch.tensor(img, dtype=torch.float32), torch.tensor(mask, dtype=torch.long)


In [27]:
full_dataset = SegmentationDataset(IMAGES_DIR, MASKS_DIR, image_size=512, num_classes=4)

# 80/20 split
train_size = int(0.8 * len(full_dataset))
val_size   = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

BATCH_SIZE = 4
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")
xb, yb = next(iter(train_loader))
print("Batch shapes -> images:", tuple(xb.shape), "masks:", tuple(yb.shape))  # expect [B,3,512,512], [B,512,512]


Train samples: 185, Val samples: 47
Batch shapes -> images: (4, 3, 512, 512) masks: (4, 512, 512)


In [28]:
class AttentionUpsample(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super().__init__()
        # Project input and skip connections
        self.conv_in = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv_skip = nn.Conv2d(skip_channels, out_channels, kernel_size=1)

        # Attention map
        self.attn = nn.Sequential(
            nn.Conv2d(out_channels * 2, out_channels, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, 1, kernel_size=1),
            nn.Sigmoid()
        )

        self.conv_out = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x, skip):
        # 1. Align size
        x = nn.functional.interpolate(x, size=skip.shape[2:], mode="bilinear", align_corners=False)

        # 2. Project features
        x_proj = self.conv_in(x)
        skip_proj = self.conv_skip(skip)

        # 3. Compute attention mask
        attn_map = self.attn(torch.cat([x_proj, skip_proj], dim=1))

        # 4. Weighted fusion
        out = attn_map * x_proj + (1 - attn_map) * skip_proj
        out = self.conv_out(out)
        return out


In [29]:
import torch
import torch.nn as nn
from transformers import SegformerModel

class CustomSegFormer(nn.Module):
    def __init__(self, num_classes=4, id2label=None, label2id=None):
        super(CustomSegFormer, self).__init__()

        # === Encoder (pretrained SegFormer B2) ===
        self.encoder = SegformerModel.from_pretrained(
            "nvidia/segformer-b2-finetuned-ade-512-512",
            output_hidden_states=True
        )

        # SegFormer B2 hidden sizes
        embed_dim = [64, 128, 320, 512]
        decoder_dim = 256

        # Projections
        self.linear_c4 = nn.Conv2d(embed_dim[3], decoder_dim, kernel_size=1)
        self.linear_c3 = nn.Conv2d(embed_dim[2], decoder_dim, kernel_size=1)
        self.linear_c2 = nn.Conv2d(embed_dim[1], decoder_dim, kernel_size=1)
        self.linear_c1 = nn.Conv2d(embed_dim[0], decoder_dim, kernel_size=1)

        # Attention-guided upsampling blocks
        self.agu3 = AttentionUpsample(decoder_dim, decoder_dim, decoder_dim)
        self.agu2 = AttentionUpsample(decoder_dim, decoder_dim, decoder_dim)
        self.agu1 = AttentionUpsample(decoder_dim, decoder_dim, decoder_dim)

        # Final classifier
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Conv2d(decoder_dim, num_classes, kernel_size=1)

    def forward(self, pixel_values):
        # === Encoder ===
        outputs = self.encoder(pixel_values)
        c1, c2, c3, c4 = outputs.hidden_states  # already (B, C, H, W)

        # Project to same decoder_dim
        c1, c2, c3, c4 = self.linear_c1(c1), self.linear_c2(c2), self.linear_c3(c3), self.linear_c4(c4)

        # === Decoder with AGU ===
        x = self.agu3(c4, c3)  # fuse high-level with C3
        x = self.agu2(x, c2)   # fuse with C2
        x = self.agu1(x, c1)   # fuse with C1

        # Final head
        x = self.dropout(x)
        logits = self.classifier(x)

        # Restore to original image size
        logits = nn.functional.interpolate(logits, size=pixel_values.shape[2:], mode="bilinear", align_corners=False)

        return logits


In [30]:
import torch

if torch.cuda.is_available():
    print("✅ GPU is available!")
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("❌ No GPU detected.")


✅ GPU is available!
GPU Name: Tesla T4


In [31]:
# === Usage ===
id2label = {0: "Urban", 1: "Water", 2: "Vegetation", 3: "Others"}
label2id = {v: k for k, v in id2label.items()}

model = CustomSegFormer(num_classes=4, id2label=id2label, label2id=label2id)
print(model)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/110M [00:00<?, ?B/s]

CustomSegFormer(
  (encoder): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
      )


model.safetensors:   0%|          | 0.00/110M [00:00<?, ?B/s]

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# id2label = {0:"Urban", 1:"Water", 2:"Vegetation", 3:"Others"}
# label2id = {v:k for k,v in id2label.items()}

# model = SegformerForSemanticSegmentation.from_pretrained(
#     "nvidia/segformer-b0-finetuned-ade-512-512",
#     num_labels=4,
#     id2label=id2label,
#     label2id=label2id,
#     ignore_mismatched_sizes=True,  # new head (4 classes) initialized
# )
model.to(device)

# Optim & training settings
optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-4)
num_epochs = 10


In [33]:
def pixel_accuracy(preds, labels):
    # preds: [B,H,W] (class ids), labels: [B,H,W]
    correct = (preds == labels).sum().item()
    total = labels.numel()
    return correct / total

def mean_iou(preds, labels, num_classes=4):
    # preds, labels: [B,H,W]
    ious = []
    for cls in range(num_classes):
        pred_c = (preds == cls)
        label_c = (labels == cls)
        inter = (pred_c & label_c).sum().item()
        union = (pred_c | label_c).sum().item()
        if union == 0:
            ious.append(float("nan"))  # ignore if class not present in both
        else:
            ious.append(inter / union)
    return np.nanmean(ious)


In [34]:
print(f"Train samples: {len(train_dataset)} ({len(train_loader)} batches of size {BATCH_SIZE})")
print(f"Val samples:   {len(val_dataset)} ({len(val_loader)} batches of size {BATCH_SIZE})")


Train samples: 185 (47 batches of size 4)
Val samples:   47 (12 batches of size 4)


In [ ]:
train_losses, val_losses = [], []
train_accs, val_accs = [], []
train_ious, val_ious = [], []

# Define the loss function
criterion = nn.CrossEntropyLoss()

for epoch in range(1, num_epochs+1):
    # ---- Train ----
    model.train()
    tr_loss, tr_acc, tr_iou = 0.0, 0.0, 0.0
    for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs} - Training"):
        imgs, masks = imgs.to(device), masks.to(device)

        # out = model(pixel_values=imgs, labels=masks)   # CrossEntropy inside
        # loss = out.loss
        # logits = out.logits                            # [B, num_classes, H, W]

        logits = model(pixel_values=imgs)             # [B, num_classes, H, W]
        loss = criterion(logits, masks)               # Calculate loss manually

        preds = torch.argmax(logits, dim=1)            # [B,H,W]

        # Resize predictions to match mask size for metric calculation
        preds_resized = torch.nn.functional.interpolate(preds.unsqueeze(1).float(),
                                                        size=masks.shape[-2:],
                                                        mode='nearest').squeeze(1).long()


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss += loss.item()
        tr_acc  += pixel_accuracy(preds_resized, masks)
        tr_iou  += mean_iou(preds_resized, masks, num_classes=4)

    tr_loss /= len(train_loader)
    tr_acc  /= len(train_loader)
    tr_iou  /= len(train_loader)

    train_losses.append(tr_loss)
    train_accs.append(tr_acc)
    train_ious.append(tr_iou)

    # ---- Validate ----
    model.eval()
    va_loss, va_acc, va_iou = 0.0, 0.0, 0.0
    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc="Validating"):
            imgs, masks = imgs.to(device), masks.to(device)
            # out = model(pixel_values=imgs, labels=masks)
            # va_loss += out.loss.item()
            logits = model(pixel_values=imgs)
            va_loss += criterion(logits, masks).item()


            # logits = out.logits  # Remove this line
            preds = torch.argmax(logits, dim=1)
            # Resize predictions to match mask size for metric calculation
            preds_resized = torch.nn.functional.interpolate(preds.unsqueeze(1).float(),
                                                            size=masks.shape[-2:],
                                                            mode='nearest').squeeze(1).long()


            va_acc += pixel_accuracy(preds_resized, masks)
            va_iou += mean_iou(preds_resized, masks, num_classes=4)

    va_loss /= len(val_loader)
    va_acc  /= len(val_loader)
    va_iou  /= len(val_loader)

    val_losses.append(va_loss)
    val_accs.append(va_acc)
    val_ious.append(va_iou)

    print(f"Epoch {epoch}/{num_epochs} | "
          f"Train Loss: {tr_loss:.4f}  Val Loss: {va_loss:.4f}  |  "
          f"Train Acc: {tr_acc*100:.2f}%  Val Acc: {va_acc*100:.2f}%  |  "
          f"Train mIoU: {tr_iou:.4f}  Val mIoU: {va_iou:.4f}")

Epoch 1/10 - Training:  60%|█████▉    | 28/47 [00:16<00:10,  1.84it/s]

In [ ]:
epochs = range(1, num_epochs+1)
plt.figure(figsize=(16,5))

plt.subplot(1,3,1)
plt.plot(epochs, train_losses, label="Train")
plt.plot(epochs, val_losses, label="Val")
plt.title("Loss"); plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()

plt.subplot(1,3,2)
plt.plot(epochs, [a*100 for a in train_accs], label="Train")
plt.plot(epochs, [a*100 for a in val_accs], label="Val")
plt.title("Pixel Accuracy"); plt.xlabel("Epoch"); plt.ylabel("Accuracy (%)"); plt.legend()

plt.subplot(1,3,3)
plt.plot(epochs, train_ious, label="Train")
plt.plot(epochs, val_ious, label="Val")
plt.title("Mean IoU"); plt.xlabel("Epoch"); plt.ylabel("mIoU"); plt.legend()

plt.show()


In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import torch

# Color map for 5 classes
COLOR_MAP = {
    0: (255, 255,   0),   # Barren -> Yellow
    1: (255,   0,   0),   # Urban  -> Red
    2: (  0,   0, 255),   # Water  -> Blue
    3: (  0, 255,   0),   # Vegetation -> Green
    4: (128, 128, 128)    # Others -> Gray
}

def decode_color(mask_np):
    """Map class indices to RGB colors for visualization"""
    h, w = mask_np.shape
    rgb = np.zeros((h, w, 3), dtype=np.uint8)
    for cls, color in COLOR_MAP.items():
        rgb[mask_np == cls] = color
    return rgb

# -------------------------------
# Inference for a specific index
# -------------------------------
model.eval()
idx = 6   # <-- choose any index here

# Make sure dataset returns correct pair
img, true_mask = val_dataset[idx]   # img: [3,512,512], true_mask: [512,512]
img_batch = img.unsqueeze(0).to(device)

with torch.no_grad():
    outputs = model(pixel_values=img_batch)
    pred_mask = torch.argmax(outputs, dim=1).squeeze(0).cpu().numpy()  # Changed outputs.logits to outputs

# Convert tensors back for visualization
true_mask = true_mask.cpu().numpy()
img_vis = img.cpu().numpy()

# Undo normalization for display
mean = np.array([0.485, 0.456, 0.406]).reshape(3,1,1)
std  = np.array([0.229, 0.224, 0.225]).reshape(3,1,1)
img_vis = (img_vis * std + mean).clip(0,1)
img_vis = np.transpose(img_vis, (1,2,0))  # CHW -> HWC

# -------------------------------
# Show results
# -------------------------------
plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_vis); plt.title(f"Original Image #{idx}"); plt.axis("off")
plt.subplot(1,3,2); plt.imshow(decode_color(true_mask)); plt.title("Ground Truth Mask"); plt.axis("off")
plt.subplot(1,3,3); plt.imshow(decode_color(pred_mask)); plt.title("Predicted Mask"); plt.axis("off")
plt.show()